In [427]:
# Importation librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category = UserWarning, module = 'openpyxl')

**Note**:
* Est-ce que product_id dans df_erp et df_liaison contiennent bien strictement les mêmes données ?

<img src='./Images/logo_Bottleneck_sans_fond.png' width=180px />

# Optimisez la gestion des données d'une boutique avec Python

## Table des matières

## Rapprochement des exports

### Importations des données

Dans un premier temps, nous chargeons les données de chaque fichier Excel dans des *DataFrame* séparés puis nous affichons les cinq premières lignes ainsi que la taille des jeux de données.

In [428]:
df_erp = pd.read_excel('./Datasets/erp.xlsx')
df_liaison = pd.read_excel('./Datasets/liaison.xlsx')
df_web = pd.read_excel('./Datasets/web.xlsx')

In [429]:
df_erp.head()

,product_id,onsale_web,price,stock_quantity,stock_status
0,3847,1,24.2,0,outofstock
1,3849,1,34.3,0,outofstock
2,3850,1,20.8,0,outofstock
3,4032,1,14.1,0,outofstock
4,4039,1,46.0,0,outofstock


In [430]:
df_erp.shape

(825, 5)

Les données de l'ERP contiennent 825 produits renseignés sur 5 variables:
* « product_id » : le code du produit;
* « onsale_web » : indique si le produit est disponible à la vente en ligne;
* « price » : le prix du produit;
* « stock_quantity » et « stock_status » qui donne des informations sur l'état des stocks.

In [431]:
df_liaison.head()

,product_id,id_web
0,3847,15298
1,3849,15296
2,3850,15300
3,4032,19814
4,4039,19815


In [432]:
df_liaison.shape

(825, 2)

Les données du fichier de liaison contiennent également 825 produits avec 2 variables:
* « product_id » : le code du produit issu de l'ERP;
* « id_web » : le code du produit issu du CRM.

Nous savons que « id_web » correspont en fait au SKU du CRM donc nous allons tout de suite renommer la colonne.

In [433]:
df_liaison = df_liaison.rename(columns = {'id_web':'sku'})

In [434]:
df_web.head()

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,...,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,bon-cadeau-25-euros,0,0,0,0.0,10.0,taxable,NaN,1.0,2018-06-01 13:53:46,...,bon-cadeau-de-25-euros,2018-06-01 14:13:57,2018-06-01 12:13:57,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
1,15298,0,0,0,0.0,6.0,taxable,NaN,2.0,2018-02-08 12:58:52,...,pierre-jean-villa-saint-joseph-preface-2018,2019-12-30 09:30:29,2019-12-30 08:30:29,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
2,15296,0,0,0,0.0,0.0,taxable,NaN,2.0,2018-02-08 13:49:41,...,pierre-jean-villa-saint-joseph-tilde-2017,2019-12-21 09:00:17,2019-12-21 08:00:17,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
3,15300,0,0,0,0.0,0.0,taxable,NaN,2.0,2018-02-08 14:08:36,...,pierre-jean-villa-croze-hermitage-accroche-coe...,2020-06-26 18:15:03,2020-06-26 16:15:03,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0
4,19814,0,0,0,0.0,3.0,taxable,NaN,2.0,2018-02-09 14:01:05,...,pierre-jean-villa-igp-gamine-2018,2020-01-04 16:36:01,2020-01-04 15:36:01,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0


In [435]:
df_web.shape

(1513, 28)

Cette fois, pour chaque produit vendu, il y a un peu plus d'informations disponibles.<br>
Il y a 28 variables, mais beaucoup semblent vides ou ou ne semblent comporter que des 0. Nous allons voir par la suite si il est possible d'en supprimer quelques unes.<br>
Le nombre de ventes est supérieur au nombre de produits dans le *DataFrame* `df_erp` mais cela semble logique car un même produit peut très bien avoir été vendu plusieurs fois.

### Vérification des type de données

Les trois fichiers sont correctement importés. Vérifions le type des données présentes dans les *DataFrame* en commençant par les données de l'ERP.

In [436]:
df_erp.dtypes

product_id          int64
onsale_web          int64
price             float64
stock_quantity      int64
stock_status       object
dtype: object

La variable « product_id » est qualitative, nous allons donc la convertir en objet.<br>
La variable « onsale_web » semble être booléenne, nous allons la convertir à conditon qu'elle ne comprenne bien que les valeurs 0 ou 1.

In [437]:
# Vérification valeurs présentes dans 'onsale_web'
verification_valeurs = df_erp['onsale_web'].isin([0,1]).any()
print(f'La variable \'onsale_web\' ne contient bien que les valeurs 0 ou 1 : {verification_valeurs}.')
del verification_valeurs

La variable 'onsale_web' ne contient bien que les valeurs 0 ou 1 : True.


In [438]:
# Conversion type
df_erp['product_id'] = df_erp['product_id'].astype(object)
df_erp['onsale_web'] = df_erp['onsale_web'].astype(bool)

Passons ensuite aux données du fichier du liaison.

In [439]:
df_liaison.dtypes

product_id     int64
sku           object
dtype: object

La variable « product_id » est qualitative, nous allons donc la convertir en objet.

In [440]:
# Conversion type
df_liaison['product_id'] = df_liaison['product_id'].astype(object)

Et pour finir, les données du CRM. Nous avons vu précédemment que le nombre de variables était assez important. Donc avant de commencer, nous allons vérifier si il est possible d'en supprimer quelques-unes. Pour cela, nous allons utiliser la méthode `.describe()`. Cette dernière va nous donner des informations sur les variables quantitatives du *DataFrame*.

In [441]:
df_web.describe()

,virtual,downloadable,rating_count,average_rating,total_sales,tax_class,post_author,post_content,post_password,post_content_filtered,post_parent,menu_order,comment_count
count,1513.0,1513.0,1513.0,1430.0,1430.000000,0.0,1430.000000,0.0,0.0,0.0,1430.0,1430.0,1430.0
mean,0.0,0.0,0.0,0.0,4.006993,NaN,1.998601,NaN,NaN,NaN,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,8.510559,NaN,0.037385,NaN,NaN,NaN,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.000000,NaN,1.000000,NaN,NaN,NaN,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.000000,NaN,2.000000,NaN,NaN,NaN,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,1.000000,NaN,2.000000,NaN,NaN,NaN,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,4.000000,NaN,2.000000,NaN,NaN,NaN,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,96.000000,NaN,2.000000,NaN,NaN,NaN,0.0,0.0,0.0


On constate que plusieurs variables ont pour seule valeur 0 et d'autres n'ont aucune valeur. Toutes ces variables peuvent donc être supprimées sans risque.

In [442]:
# Création liste colonnes et suppression
variables_a_supprimer = (
    set (df_web.select_dtypes(np.number).columns[df_web.mean(numeric_only = True) == 0]) # Colonnes avec valeur uniquement 0 (donc moyenne = 0)
    | set(df_web.columns[df_web.count() == 0]) # Union avec colonnes vides
)
df_web = df_web.drop(variables_a_supprimer, axis = 1)
del variables_a_supprimer

In [443]:
df_web.dtypes

sku                          object
total_sales                 float64
tax_status                   object
post_author                 float64
post_date            datetime64[ns]
post_date_gmt        datetime64[ns]
post_title                   object
post_excerpt                 object
post_status                  object
comment_status               object
ping_status                  object
post_name                    object
post_modified        datetime64[ns]
post_modified_gmt    datetime64[ns]
guid                         object
post_type                    object
post_mime_type               object
dtype: object

Tout semble correct à l'exception de la variable « post_author » qui est en fait qualitative et que nous allons convertir.

In [444]:
# Conversion type
df_web['post_author'] = df_web['post_author'].astype(object)

-------

In [445]:
test_product_id = not (set(df_erp['product_id'].unique()) - set(df_liaison['product_id'].unique()))
print(f'Les « product_id » contenus dans « df_erp » et « df_liaison » sont les mêmes : {test_product_id}')
del test_product_id

Les « product_id » contenus dans « df_erp » et « df_liaison » sont les mêmes : True


Les codes produits sont les mêmes dans l'ERP et dans le fichier de liaison. Vérifions maintenant si des produits non destinés à la vente sur internet ont un SKU.

In [446]:
onsale_web_false = not (set(df_erp.loc[df_erp['onsale_web'] == False, 'product_id']) - set(df_liaison.loc[df_liaison['sku'].isna(), 'product_id']))
print(f'Quand « onsale_web » dans « df_erp » est faux, « sku » est vide dans « df_liaison » : {onsale_web_false}')
del onsale_web_false

Quand « onsale_web » dans « df_erp » est faux, « sku » est vide dans « df_liaison » : False


Voyons également si il manque le SKU pour des produits censés être vendus en ligne.

In [447]:
onsale_web_true = not (set(df_erp.loc[df_erp['onsale_web'] == True, 'product_id']) - set(df_liaison.loc[~df_liaison['sku'].isna(), 'product_id']))
print(f'Quand « onsale_web » dans « df_erp » est vrai, « sku » est renseigné dans « df_liaison » : {onsale_web_true}')
del onsale_web_true

Quand « onsale_web » dans « df_erp » est vrai, « sku » est renseigné dans « df_liaison » : False


In [448]:
len(set(df_erp.loc[df_erp['onsale_web'] == False, 'product_id']) - set(df_liaison.loc[df_liaison['sku'].isna(), 'product_id']))

20

In [449]:
len(set(df_erp.loc[df_erp['onsale_web'] == True, 'product_id']) - set(df_liaison.loc[~df_liaison['sku'].isna(), 'product_id']))

3